In [1]:
from requests import get
from json import loads
from time import sleep
from typing import List
from borsdata_sdk.models.Instrument import Instrument
from borsdata_sdk.models.Market import Market
import pandas as pd
from http import HTTPStatus
from borsdata_sdk import BorsdataAPI
import borsdata_sdk
import requests
from pandas.io.json import json_normalize

In [2]:
auth_key = 'f4e8e3c7c46c4c34ae238a1822863410'

RATE_LIMIT = 429
borsdata = BorsdataAPI(auth_key)
instruments = borsdata.get_instruments()

In [3]:
my_list = []
for i in range(len(instruments)):
    tmp_list = [instruments[i].insId,instruments[i].name,instruments[i].countryId,instruments[i].sectorId,instruments[i].marketId,instruments[i].instrument]
    my_list.append(tmp_list)

instruments_df=pd.DataFrame(my_list,columns=['insID','Name','countryId','sectorId', 'marketId', 'instrument']) 
# Rename the lists to the real names
# MarketId: 1 = Large Cap, 2 = Mid Cap, 3=Small Cap, 4=First North
swe_stock_main = instruments_df[(instruments_df['countryId'] == 1) & (instruments_df['instrument'] == 0) & (instruments_df['marketId'].isin([1,2,3, 4]))]



In [4]:
swe_stock_main

,insID,Name,countryId,sectorId,marketId,instrument
0,2,AAK,1,2.0,1,0
1,3,ABB,1,5.0,1,0
2,6,Active Biotech,1,4.0,3,0
3,7,Addnode,1,6.0,2,0
4,8,Addtech,1,5.0,1,0
...,...,...,...,...,...,...
1861,2332,Viva Wine,1,2.0,4,0
1864,2335,Case Group,1,1.0,4,0
1865,2336,Nordic Asia,1,1.0,4,0
1875,2346,MTI Investment,1,1.0,4,0


In [5]:
### LIST OF ALL KPIS

kpis_list = ["151/1year/return","151/6month/return","151/3month/return","2/last/latest",
             '3/last/latest', "4/last/latest"]



kpis_names = ["1year_return",'6month_return',"3month_return" ,"pe", 'ps', 'pb']

In [16]:
if len(kpis_list) == len(kpis_names):

    kpi_df = pd.DataFrame()

    for kpi in kpis_list:
        URL = "https://apiservice.borsdata.se/v1/instruments/kpis/"+ kpi +"?authKey=f4e8e3c7c46c4c34ae238a1822863410"
        response = requests.get(URL)
        tmp_df = json_normalize(response.json(), record_path ='values')

        tmp_df.rename(columns={'n': 
                           pd.DataFrame(response.json())['group'].unique()[0] +"_"+ 
                           pd.DataFrame(response.json())['calculation'].unique()[0]}, inplace=True)
        tmp_df = tmp_df.set_index(['i'])
        tmp_df = tmp_df.drop(['s'], axis = 1)
        kpi_df = pd.concat([kpi_df, tmp_df], axis=1)
        
    
else:
    print("Unequal length of lists")


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


In [14]:
kpi_df

,1year_return,6month_return,3month_return,last_latest,last_latest,last_latest
i,,,,,,
2,15.861027,-1.540437,3.508772,35.323479,1.512535,4.430882
3,46.953556,14.091964,17.954857,42.178768,3.034897,6.212935
6,-33.789471,-17.670156,-9.365993,-7.275882,44.227219,4.366382
7,45.357143,20.414202,18.313953,68.918808,3.513361,8.220419
8,71.681419,22.012579,22.784811,61.687180,4.232623,14.663910
...,...,...,...,...,...,...
2345,NaN,NaN,NaN,NaN,NaN,NaN
2346,NaN,NaN,NaN,NaN,NaN,NaN
2347,NaN,NaN,NaN,NaN,NaN,NaN


### Get data not available from the API

Data we cannot get
- Yahoo ticker
- Volume 50 day avergae
- Ticker